## Draw Most Salient Objects based on bottom-up saliency in GazeCom Videos

In [ ]:

videos_run =[s for s in range(0,17)]
num_frames = 595
for v in tqdm(videos_run, desc="videos"):#len(video_path_list)
    videoloc = data_path_list[v] #video_path_list[v]
    print(videoloc)
    video_name = os.path.splitext(os.path.basename(videoloc))[0][:-5]

    most_salient_objects_path = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/lowlevel-salient-objects/'
    mask_path = most_salient_objects_path + video_name + '_lowlevel-salient-objects.npy'
    salient_object_ids = np.load(mask_path,allow_pickle=True)
    panoptic_seg = np.load(data_path_list[v], allow_pickle=True)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    in_file = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_data.nosync/LUEBECK/' + video_name + '.mpg'
    out_file = most_salient_objects_path + video_name + '.avi'

    video_in = cv2.VideoCapture(in_file)
    video_out = cv2.VideoWriter(out_file, fourcc, 30, (1280,720))
    success,img = video_in.read()
    count = 0
    while (success and count < 595):
        #resized = cv2.resize(salient_object_mask[count,:,:], (1280,720), interpolation = cv2.INTER_AREA)
        for i in range(len(salient_object_ids[count])):
            segment_id = salient_object_ids[count][i]
            mask = (panoptic_seg[count].numpy()==segment_id).astype(np.uint8)
            if (np.count_nonzero(mask)!=0):
                contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                cv2.drawContours(img, contours, -1, (255, 0, 0), 3) 
                # compute the center of the contour
                for c in contours:
                    M = cv2.moments(c)
                    if M['m00']!=0:
                    
                        cX = int(M["m10"] / M["m00"])
                        cY = int(M["m01"] / M["m00"])
                        # draw the contour and center of the shape on the image
                        cv2.putText(img, str(i), (cX, cY),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        count += 1
        video_out.write(img)
        success,img = video_in.read()
    video_out.release()
    video_in.release()
